In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import json
import malaya
from malaya.dictionary import *

In [3]:
language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-v1')

In [4]:
deep_model = malaya.language_detection.deep_model()

2023-06-05 15:53:37.072679: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 15:53:37.080351: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-05 15:53:37.080377: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2023-06-05 15:53:37.080380: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2023-06-05 15:53:37.080571: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.182.3
2023-06-05 15:53:37.080587: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported vers

In [9]:
%%time

deep_model.predict_proba(['gk bisa ni'])[0]['ind']

CPU times: user 1.6 ms, sys: 0 ns, total: 1.6 ms
Wall time: 1.59 ms


1.0

In [10]:
malay_words = MALAY_WORDS | CAMBRIDGE_MALAY_WORDS | KAMUS_DEWAN_WORDS | DBP_WORDS
malay_words |= negeri | city | country | daerah | parlimen | adun
len(malay_words)

65823

In [11]:
english_words = ENGLISH_WORDS
len(english_words)

234177

In [12]:
english_minus = english_words - malay_words
len(english_minus)

231316

In [13]:
english_minus = {i for i in english_minus if 'haha' not in i and i != 'rt' and 'yeay' not in i and\
                'yes' not in i and 'ooo' not in i and 'insha' not in i and 'huhu' not in i and\
                'insya' not in i and 'hew' not in i and 'uwuu' not in i and\
                 'wkwk' not in i and 'hoho' not in i and 'babi' not in i and\
                 'meow' not in i and 'aiii' not in i and 'alham' not in i and 'mashaa' not in i\
                 and i not in ['takda', 'cer']}
len(english_minus)

231117

In [14]:
import re

def language_detection_textcleaning(string):
    string = re.sub(
        'http\\S+|www.\\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )

    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
    string = string.replace('\n', '').replace('\t', '')

    string = re.sub(
        '[0-9!@#$%^&*()_\\-+{}|\\~`\'";:?/.>,<]', ' ', string, flags=re.UNICODE
    )
    string = re.sub(r'[ ]+', ' ', string).strip()

    return string.lower()


In [15]:
import itertools

def _pad_sequence(
    sequence,
    n,
    pad_left = False,
    pad_right = False,
    left_pad_symbol = None,
    right_pad_symbol = None,
):
    sequence = iter(sequence)
    if pad_left:
        sequence = itertools.chain((left_pad_symbol,) * (n - 1), sequence)
    if pad_right:
        sequence = itertools.chain(sequence, (right_pad_symbol,) * (n - 1))
    return sequence

def ngrams(
    sequence,
    n: int,
    pad_left = False,
    pad_right = False,
    left_pad_symbol = None,
    right_pad_symbol = None,
):
    sequence = _pad_sequence(
        sequence, n, pad_left, pad_right, left_pad_symbol, right_pad_symbol
    )

    history = []
    while n > 1:
        try:
            next_item = next(sequence)
        except StopIteration:
            return
        history.append(next_item)
        n -= 1
    for item in sequence:
        history.append(item)
        yield tuple(history)
        del history[0]

In [16]:
len([w for w in english_minus if len(w) > 2]), len(english_minus)

(231114, 231117)

In [17]:
def split(data):
    t = language_detection_textcleaning(data)
    splitted = t.split()
    ngs = set(splitted)
    ngs |= set([' '.join(n) for n in ngrams(splitted, 2)])
    ngs |= set([' '.join(n) for n in ngrams(splitted, 3)])
    return ngs

In [18]:
%%time

split("datang saman peniaga kecil tanpa lesen di tepi jalan tapi diri sendiri tak pakai mask lokasi berhadapan masjid beijing rantau panjang") & english_minus

CPU times: user 30 µs, sys: 14 µs, total: 44 µs
Wall time: 45.8 µs


{'mask'}

In [123]:
!tail -n 3000 filter-twitter-malay-rojak-ms.jsonl

"kpal ini nah perlu mata jeli buat dpt mment"
"ketereh bodoh abadi ni"
"bayangkan bagi k laptop umno pulak geram dia bagi k pas bagi k itu belom kayangan lagi bagi k sebab geraaamm sangat tak kurang k warga muo dapat manfaat kan bagus gitu maju beb muo kalau gitu"
"kahkahkah sabar jim sabar"
"kahkahkah sabar jim sabar"
"rt untuk rezeki heffa"
"rt untuk rezeki heffa"
"kiri lelaki uk kanan lelaki terengganu"
"bomboclaat"
"meh sepang heheheh"
"makan sbb kedai aate i sebelah tu je so tayah bungkus"
"makan sbb kedai aate i sebelah tu je so tayah bungkus"
"yang x pakai topeng ni sape lak nak saman ke kira setel belakang"
"kiri lelaki uk kanan lelaki terengganu"
"demi sesuap nasi"
"mengerikan negeri yg dikuasai yg dikuasai oligarki berbasis korupsi sumber daya alam ini pulau sangihe jd konsesi tambang pulau padar konsesi resort ha hutan di puncak kota labuan bajo konsesi hotel dan banyak lagi kasus serupa negeri ini tdk sedang baik saja"
"kiri lelaki uk kanan lelaki terengganu"
"kiri lelaki u

In [129]:
eng_words = {
    'i m at', 'admin info',
}

id_words = {
'njneng', 'ngakakk', 'ngakak', 'pemberantasan', 'dilemparibatu', 'ricuh',
'ngantongin', 'buek tagak', 'bgt cok', 'ngobok', 'kasian iya', 'pasya sik', 'sik', 'dong', 'goblok',
'kau sik', 'bni sik', 'ngajarin', 'gausah', 'komnas', 'twk bukan', 'kagakkkk sik', 'kagak', 'kagakk sik',
'kagakk', 'sifit', 'siklepas', 'bait sik', 'disuruh nyebut', 'konsul', 'jum sik', 'apa sik', 'tindaklanjuti',
'pariwisata', 'kadin', 'iki ngerat', 'batu giok', 'kyk yg', 'aja gitu', 'gada sik', 'waheguru',
'pupuk kaltim', 'simpen', 'haluu bgt', 'halu bgt', 'haloo bgt', 'helo bgt', 'wadah baskom',
'ndak ku', 'dah sik', 'cpek bgt', 'yg mw', 'pancasakti', 'stie indonesia', 'pgn jd', 'gw orangnya',
'pke batu', 'sy butuh', 'sik yol', 'parkiran', 'di dket', 'sempol trus', 'bakpia pathok', 'konpembalit',
'seko bogor', 'sik di', 'sik banter', 'ak kngen', 'kngen', 'putussibau', 'nonton bola', 'pantes',
'emosian', 'ga ngakuin', 'gw terus', 'kretek', 'kantor', 'posyandu', 'ngalamin', 'purnawirawan',
'temen', 'nyeletuk', 'semalam aj', 'cancik bgt', 'cantik bgt', 'wakapolres', 'batu kompol', 'apel pagi',
'suharsono', 'cuma karna', 'tetap aja', 'divonis', 'pariwi', 'pebisnis', 'gws syanggakuy',
'sik gm', 'ndak tahu', 'ngerebus', 'kluster sik', 'nya bikin', 'wib cikunir', 'ada sik', 'di vidio',
'civitas', 'sekte', 'kejametan', 'khawatir', 'ngomong', 'dah kyknya', 'kyknya', 'kakanwil kemenkumham',
'ngoco', 'sik deh', 'emg ccook', 'emg cocok', 'bgt njr', 'ga asa', 'latihan sik',
'emg jeli', 'jeli bgt', 'eks fpi', 'emng berharap', 'gak sopan', 'bngt ya', 'ndak tahu', 'nggak tahu',
'sik lejen', 'santo martinus', 'katolik santo', 'kantongin', 'lah trus', 'lah terus','diruwat',
'pt aneka', 'tbk antam', 'batu rebus', 'yaa alloh', 'ya alloh', 'iki lawas', 'sdh brp', 'sakuin',
'pakpak', 'sekda', 'manusiawi', 'mengobati', 'cuka apel', 'giniin', 'ini benar'
'emank nya', 'nasbung', 'kadrun', 'sik lamak', 'sarapan sik', 'putusan ya', 'lw ngpa', 'ngpa sik',
'soal ppn', 'ppn sembako', 'kan gw', 'tl mulai', 'mulai sik', 'batu bara', 'blm sik',
'gw optimis', 'ktk sik', 'pegawaine', 'kaga sik', 'kantongin', 'konsolidasi rupiah',
'kenapasih', 'gitu aja', 'jempol', 'kelaminmu lanang', 'sik ramah', 'kami membutuhkan', 'saya membutuhkan',
'kenapa sik', 'soekarno pki', 'konsekuen', 'progresifrevolusioner', 'sik temu', 'presiden widodo',
'duani perciapan', 'dipake', 'pesta dansa', 'kusno', 'sekonyong konyong', 'ngemalesin', 'bnget',
'iya neh', 'ya emang', 'mengedukasi', 'air keras', 'gapapaaa', 'gapapaa', 'gapapa',
'sik ge', 'ge bubar eh', 'booster mudik tenang', 'batu akik', 'senter', 'ujang marbot', 'cowok',
'dimentokin', 'stadion', 'mass cek en', 'gemesh bgt', 'si mas', 'maaf mas', 'batu cepio',
'sik wbu', 'mau nyari', 'bolaang', 'listrik', 'konversi', 'ga muat', 'lancar sel',
'reklamasi', 'dp rupiah', 'kasus mu', 'maguwoharjo', 'batu akik', 'gabisa', 'aja min',
'si cwo', 'si cwe', 'cewek', 'cwe cwo', 'lama bgt', 'trs si', 'inget dulu',
'sapi banteng', 'bangat', 'ga masuk', 'dicariin', 'eh bentar', 'gak nyangka',
'gratis','iberet','mewek','ponakan', 'dian sastro', 'kreatifitas', 'tipenya',
'fotoin', 'timnya', 'yg bner', 'mengapresiasi', 'panitia', 'pria', 'cowok', 'arek',
'sprei sarung', 'harus optimis', 'kepleset', 'liat aja', 'rame rame', 'es buah', 'itungin'
'ga sengaja', 'kaget', 'varietas','bacotan', 'malem', 'siksaan', 'subdenpom', 'polres',
'sm aj', 'wirausaha', 'ngeselin', 'pamswakarsa', 'sutet', 'suantui', 'perekonomian',
'ngefokusin', 'cerahin', 'ketauan', 'favorit', 'menuhin', 'oh sik', 'aaratan',
'jati bening', 'pasti rame', 'pemanenan', 'mending', 'polos', 'cebong',
'aja ntar', 'niup', 'bang atuy', 'kopdagrin','kuansing', 'enak jg', 'mochtar riady',
'deteksi','omongan', 'jelek', 'lunas spp', 'olesin', 'selamat sore', 'rekening', 'kondisi', 'ryamizad ryacudu',
'maruk', 'maruk', 'udh dong', 'udh naik', 'kaesang', 'kaesang pangarep', 'mikhail gorbachev',
'parkir', 'kertanegara', 'pengawa', 'resmi', 'titik sumur', 'bpk gw',
'dulu mba', 'ngga ya', 'bpk gw', 'senter', 'gapapa', 'keanearagaman', 'kemaren', 'ntar deh', 'hostingpangeran',
'skidipapap', 'baliho', 'kasih amp', 'amp berdoa', 'butuh aplikasi', 'sebenernya',
'bgt kaya', 'gini bgt', 'sholawatan', 'mengakui kesulitan', 'buwas wacana', 'kemenjadian'
'gak penting', 'doang', 'gemes', 'mati aja', 'bpk gw', 'mba mba', 'gik dh', 'dh kdk',
'ng buat', 'klasemen', 'kompetisi', 'ngopernya', 'warkop', 'sik tau', 'ng ntot', 'rijik',
'pensiunan', 'kota macet', 'akiknya', 'muaradua', 'emang', 'prokes', 'bagaimana biasanya'
'jajaran', 'komentar', 'kredibiltas', 'disuguhkan', 'titel', 'ketakjubanku', 'ngapasi',
'resep', 'nagasari', 'semua sik', 'sm aj', 'gorontalo', 'kapan', 'geonusantara', 'mabes',
'terus waktunya', 'jatibening', 'si mba', 'rekapitulasi', 'provinsi', 'itungan', 'tranding',
'ngetag', 'ngobrol', 'dipukulin', 'gakda', 'ngakak', 'cincing sik', 'penyeberangan', 'prakiraan'
'ndk nian', 'kancamu', 'lanang', 'donk', 'ndakra', 'es batu', 'kemenperin', 'panen', 'badarawuhi',
'sangkuriang', 'girimargo', 'babinsa', 'selogono', 'bahwa', 'katapat', 'asik', 'bangettt', 'banget', 'bangett',
'wisata', 'religi', 'gw kira', 'beneran', 'kabupaten', 'pawangnya', 'anjir',
'dipake','dong', 'dapet', 'udah', 'ngapa', 'napa sik', 'pake',
'medali', 'join yuk', 'ruteng', 'keuskupan', 'dirik sik',
'karena', 'nanya', 'ne gaya', 'terjadi nih', 'krna', 'mau ikutan', 'dikit', 'yuk',
'dong', 'gak sik', 'banyakin', 'masa iya', 'bgt sik', 'senin', 'nangis aja', 'ngerti anjir',
'ngerasain', 'kesurupan', 'gatau knp', 'masi minat', 'udh jual',
'ga bakat', 'g lo', 'sik pinter', 'ganonton', 'makasih', 'teruntuk',
'enggak', 'nyesel', 'gtu donk', 'kontolnya', 'gabaik', 'seneng', 'kalo',
'mau nya', 'nda pg', 'kangen', 'bedekan', 'capek', 'editannya', 'burik',
'rendem', 'aja dulu', 'opo sik', 'kewan', 'manjur', 'keluarin', 'demen', 'sirbal',
'dijambak', 'followan', 'istirahat', 'ancur bgt', 'kue khas', 'keukarah',
'g maoooo', 'g maoo', 'bener', 'maukah', 'gimana', 'ad loh', 'gatau', 'kepedean',
'omongin', 'kecer', 'teruntuk', 'mau liat', 'kalo mau', 'laskar', 'menerobos',
'pemukiman', 'kayak hati', 'temennya', 'temennye', 'maksain', 'jempol', 'gemeteran',
'nggk', 'ga nih', 'kagak', 'aja lo', 'sungkei', 'gada kan', 'gapake', 'kasur',
'sedih bgt', 'gaksi', 'cewe luju', 'ya sama', 'ormas', 'ulur', 'gakk tau', 'skmok',
'boleh sik', 'keturon', 'seng', 'ndak ke', 'bayangin', 'nder', 'setim', 'ngapain',
'entlek', 'mlungu', 'ndodok', 'sik nek', 'bangeeeettt', 'banget', 'betolah',
'sadar', 'bgt ya', 'pastok', 'kira kamu', 'keren', 'napas', 'permen', 'nanggung',
'ngasi', 'matane', 'picek', 'kabeh', 'gamau', 'pesimis', 'sertif', 'ngando',
'ga atasan', 'ga bawahan', 'benas', 'nek sik', 'gini ya', 'biarawati',
'dipike', 'buset', 'bjingan', 'ngebandingin', 'pantes', 'batu sotoy',
'emang', 'temennya', 'begok', 'batu doang', 'ngarep', 'ngomog', 'ngomong',
'ngentot', 'ditutupin', 'ngelunjak', 'aparat', 'sama lo', 'bintitan', 'keributan',
'bgt lo', 'diselingkuhin', 'pemberantasan', 'mafiatanah', 'boyamin kpk', 'komisi',
'konsorsium', 'mereka aja', 'terkontaminasi', 'lebih gede', 'oknum tni', 'pengusaan'
}

rejected = {
'orang sik', 'sik kedah', 'mahsala',
'icuk sugiarto', 'bachen hain', 'sab hukmaran', 'no basa basi', 'jungkook','prateek',
'adishakti jayatu', 'maha parasakthi', 'icuk sugiarto', 'rudy hartono', 'momota viktor',
'hi wahi', 'toh mangh rhy', 'mangh rhy', 'tanimbar', 'saken bera', 'elkher',
'yiwa', 'saurayi', 'abincin', 'barkha dutt', 'uliweka', 'hoga ya', 'seokjin taehyung',
'utc temp' 'jijini', 'linahitaji', 'kupamba', 'baqia', 'ganta', 'tunaona', 'kazi wengine',
'azanke', 'otlaba','yajtgz','tkqkhv','jisoo','junkyu','bolingaka', 'heya heya',
'nde','nimekusema','unatumiaga','hoye','kwaila','ogopa','watu','mengayomi','poola',
'sisrka','chori','pakray','chori','kiy','ktho','nimaz','kharche','merhameti','rahi','chargano',
'ban jate', 'batuya sariyo', 'shukriya', 'keman dersi', 'atanapi', 'sulanjanakeun', 'adaymis'
}

In [130]:
ms_open = open('filter-twitter-malay-rojak-ms.jsonl', 'w')
id_open = open('filter-twitter-malay-rojak-id.jsonl', 'w')
rojak_open = open('filter-twitter-malay-rojak-rojak.jsonl', 'w')

In [132]:
# filtered = []

# s = {'ricuh'}
# with open('filter-twitter-other.jsonl') as fopen:
#     for l in tqdm(fopen):
        
#         data = json.loads(l)
#         t = language_detection_textcleaning(data)
#         splitted = t.split()
#         ngs = set(splitted)
#         ngs = set(splitted)
#         ngs |= set([' '.join(n) for n in ngrams(splitted, 2)])
#         ngs |= set([' '.join(n) for n in ngrams(splitted, 3)])
#         if len(ngs & rejected):
#             continue
#         if len(ngs & id_words):
#             continue
#         if len(ngs & s):
#             filtered.append(t)
#             print(len(filtered))

In [133]:
filtered

[]

In [134]:
from tqdm import tqdm

with open('filter-twitter-other.jsonl') as fopen:
    for l in tqdm(fopen):
        
        data = json.loads(l)
        if language_detection.predict([data])[0] in {'eng', 'other'}:
            continue
        t = language_detection_textcleaning(data)
        splitted = t.split()
        ngs = set(splitted)
        ngs |= set([' '.join(n) for n in ngrams(splitted, 2)])
        ngs |= set([' '.join(n) for n in ngrams(splitted, 3)])
        if len(ngs & rejected):
            continue
        if len(ngs & id_words):
            f = id_open
        elif len(ngs & english_minus) or len(ngs & eng_words):
            f = rojak_open
        else:
            f = ms_open
        
        f.write(f'{json.dumps(t)}\n')
        f.flush()

20662653it [11:47, 29201.98it/s]


In [135]:
ms_open.close()
id_open.close()
rojak_open.close()

In [139]:
!wc -l filter-twitter-malay-rojak-ms.jsonl

9145589 filter-twitter-malay-rojak-ms.jsonl


In [140]:
!wc -l filter-twitter-malay-rojak-rojak.jsonl

9244863 filter-twitter-malay-rojak-rojak.jsonl


In [141]:
!wc -l filter-twitter-malay-rojak-id.jsonl

1746570 filter-twitter-malay-rojak-id.jsonl


In [142]:
!head -n 2 filter-twitter-malay-rojak-ms.jsonl

"ko mok pencak silat ne wusyu arboneng wikiyay habror daripada china aikido sekrezo wuai timol aldam buah saluk daripada maner"
"ko mok pencak silat ne wusyu arboneng wikiyay habror daripada china aikido sekrezo wuai timol aldam buah saluk daripada maner"
